In [1]:
import pandas as pd                        
from pytrends.request import TrendReq
import time


In [5]:
#pytrend = TrendReq(hl='en-US', tz=360,timeout=(10,25), proxies=['https://34.203.233.13:80',], retries=2, backoff_factor=0.1, requests_args={'verify':False} )
pytrend = TrendReq(hl='fr-FR')

In [6]:
pytrend.build_payload(kw_list=['Coronavirus'])
# Related Queries, returns a dictionary of dataframes
related_queries = pytrend.related_queries()
#related_queries.values()

In [7]:
# Related Topics, returns a dictionary of dataframes
related_topic = pytrend.related_topics()
#related_topic.values()

# Testing interest by region

In [8]:
kw_list = ["covid"]
pytrend.build_payload(kw_list, cat=0, timeframe='2020-01-01 2021-04-20', geo='FR', gprop='')


In [9]:
pytrend.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=True)


,geoCode,covid
geoName,,
Alsace,FR-A,93
Aquitaine,FR-B,93
Auvergne,FR-C,96
Basse-Normandie,FR-P,90
Bourgogne,FR-D,89
Bretagne,FR-E,96
Centre-Val de Loire,FR-F,90
Champagne-Ardenne,FR-G,83
Corse,FR-H,92


# Testing interest over time

In [10]:
kw_list = ["covid"]
pytrend.build_payload(kw_list, cat=0, timeframe='2020-01-01 2021-04-20', geo='FR-A', gprop='')

In [40]:
# Preping the region list
#colnames = ["regions","new_regions"]
df = pd.read_csv("region_trends.csv", sep=";")
df2 = df["regions"].values.tolist()
#df2.remove("Regions")
regions=df


In [31]:
df

,regions,new_regions
0,FR-A,Grand-Est
1,FR-B,Nouvelle-Aquitaine
2,FR-C,Auvergne-Rhône-Alpes
3,FR-D,Bourgogne-Franche-Comté
4,FR-E,Bretagne
5,FR-F,Centre-Val de Loire
6,FR-G,Grand-Est
7,FR-H,Corse
8,FR-I,Bourgogne-Franche-Comté
9,FR-J,Île-de-France


In [32]:
startTime = time.time()
kw_list=["psychiatre"]

dataset = []
for x in range(0,len(df2)):
#for x in range(0,2):    
     region_of_interest = df2[x]
     pytrend.build_payload(
     kw_list=kw_list,
     cat=0,
     timeframe='2020-01-01 2021-04-20',
     geo=region_of_interest)
     data = pytrend.interest_over_time()
     if not data.empty:
          data = data.drop(labels=['isPartial'],axis='columns')
          dataset.append(data)

executionTime = (time.time() - startTime)


In [33]:
result = pd.concat(dataset, axis=1)
result.columns = df2


In [53]:
#reformatage du tableau
new_result=result.reset_index().melt("date")
#utilisation des nouveaux noms de régions
new_result=new_result.merge(regions, left_on='variable', right_on="regions").drop(['variable','regions'], axis=1)
#regroupements pour avoir des lignes uniques par date et region (utilisation de la moyenne)
new_result=new_result.groupby(['date','new_regions']).mean().reset_index()
#decalage des dates au lundi (à la place du dimanche)
new_result['date']=new_result['date']+pd.Timedelta("1 days")

new_result.rename( columns={'new_regions':'region', 'value':kw_list[0]+'_feature_gtrends'}, inplace=True )

new_result.head()

,date,region,psychiatre_feature_gtrends
0,2020-01-06,Auvergne-Rhône-Alpes,46.0
1,2020-01-06,Bourgogne-Franche-Comté,64.0
2,2020-01-06,Bretagne,46.0
3,2020-01-06,Centre-Val de Loire,100.0
4,2020-01-06,Corse,0.0


In [ ]:

name = 'search_trends_' + kw_list[0]+ '_.csv'
result.to_csv(name)
print('Execution time in sec.: ' + str(executionTime))

In [75]:
sns.line()